In [6]:
import os
import numpy as np
import pandas as pd
import yfinance as yf
import talib
from datetime import datetime, timedelta
import feedparser
import nltk
from nltk.tokenize import word_tokenize
from urllib.parse import quote
from concurrent.futures import ThreadPoolExecutor

In [7]:
nltk.download('punkt_tab')

# Define the path to the ticker symbol file
ticker_symbol_file = "../ticker-symbol.txt"

# Load the Loughran-McDonald word lists
lm_dict = pd.read_csv('../Loughran-McDonald_MasterDictionary_1993-2023.csv')

print(lm_dict.columns)

# Extract sentiment word lists
negative_words = lm_dict[lm_dict['Negative'] > 0]['Word'].tolist()
positive_words = lm_dict[lm_dict['Positive'] > 0]['Word'].tolist()
uncertainty_words = lm_dict[lm_dict['Uncertainty'] > 0]['Word'].tolist()
litigious_words = lm_dict[lm_dict['Litigious'] > 0]['Word'].tolist()
strong_modal_words = lm_dict[lm_dict['Strong_Modal'] > 0]['Word'].tolist()
weak_modal_words = lm_dict[lm_dict['Weak_Modal'] > 0]['Word'].tolist()
constraining_words = lm_dict[lm_dict['Constraining'] > 0]['Word'].tolist()

print(negative_words)
print(positive_words)
print(uncertainty_words)
print(litigious_words)
print(strong_modal_words)
print(weak_modal_words)
print(constraining_words)

Index(['Word', 'Seq_num', 'Word Count', 'Word Proportion',
       'Average Proportion', 'Std Dev', 'Doc Count', 'Negative', 'Positive',
       'Uncertainty', 'Litigious', 'Strong_Modal', 'Weak_Modal',
       'Constraining', 'Complexity', 'Syllables', 'Source'],
      dtype='object')
['ABANDON', 'ABANDONED', 'ABANDONING', 'ABANDONMENT', 'ABANDONMENTS', 'ABANDONS', 'ABDICATED', 'ABDICATES', 'ABDICATING', 'ABDICATION', 'ABDICATIONS', 'ABERRANT', 'ABERRATION', 'ABERRATIONAL', 'ABERRATIONS', 'ABETTING', 'ABNORMAL', 'ABNORMALITIES', 'ABNORMALITY', 'ABNORMALLY', 'ABOLISH', 'ABOLISHED', 'ABOLISHES', 'ABOLISHING', 'ABROGATE', 'ABROGATED', 'ABROGATES', 'ABROGATING', 'ABROGATION', 'ABROGATIONS', 'ABRUPT', 'ABRUPTLY', 'ABRUPTNESS', 'ABSENCE', 'ABSENCES', 'ABSENTEEISM', 'ABUSE', 'ABUSED', 'ABUSES', 'ABUSING', 'ABUSIVE', 'ABUSIVELY', 'ABUSIVENESS', 'ACCIDENT', 'ACCIDENTAL', 'ACCIDENTALLY', 'ACCIDENTS', 'ACCUSATION', 'ACCUSATIONS', 'ACCUSE', 'ACCUSED', 'ACCUSES', 'ACCUSING', 'ACQUIESCE', 'ACQUIESCED'

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ng_mi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [8]:
def get_technical_indicator(df):
    
    #Overlap Studies

   # Bollinger Bands: Indicates overbought/oversold conditions
    df['upperband'], df['middleband'], df['lowerband'] = talib.BBANDS(df['Close'], timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)
    # Value range: [negative infinity, positive infinity]
    # Example: If the close price touches the upper band, it may be overbought (potential fall).
    #          If the close price touches the lower band, it may be oversold (potential rise).

    # Double Exponential Moving Average: Smooths price data
    df['DEMA'] = talib.DEMA(df['Close'], timeperiod=30)
    # Value range: [negative infinity, positive infinity]
    # Example: If the close price is above the DEMA, it indicates an uptrend (potential rise).
    #          If the close price is below the DEMA, it indicates a downtrend (potential fall).

    # Exponential Moving Average: Smooths price data
    df['EMA'] = talib.EMA(df['Close'], timeperiod=30)
    # Value range: [negative infinity, positive infinity]
    # Example: If the close price is above the EMA, it indicates an uptrend (potential rise).
    #          If the close price is below the EMA, it indicates a downtrend (potential fall).

    # Hilbert Transform - Instantaneous Trendline: Identifies trend direction
    df['HT_TRENDLINE'] = talib.HT_TRENDLINE(df['Close'])
    # Value range: [negative infinity, positive infinity]
    # Example: If the close price is above the HT_TRENDLINE, it indicates an uptrend (potential rise).
    #          If the close price is below the HT_TRENDLINE, it indicates a downtrend (potential fall).

    # Kaufman Adaptive Moving Average: Adjusts to market volatility
    df['KAMA'] = talib.KAMA(df['Close'], timeperiod=30)
    # Value range: [negative infinity, positive infinity]
    # Example: If the close price is above the KAMA, it indicates an uptrend (potential rise).
    #          If the close price is below the KAMA, it indicates a downtrend (potential fall).
    
    # Moving Average: Smooths price data
    df['MA'] = talib.MA(df['Close'], timeperiod=30, matype=0)
    # Value range: [negative infinity, positive infinity]
    # Example: If the close price is above the MA, it indicates an uptrend (potential rise).
    #          If the close price is below the MA, it indicates a downtrend (potential fall).
    
    # MESA Adaptive Moving Average: Adapts to market cycles
    df['MAMA'], df['FAMA'] = talib.MAMA(df['Close'], fastlimit=0.5, slowlimit=0.05)
    # Value range: [negative infinity, positive infinity]
    # Example: If the MAMA is above the FAMA, it indicates an uptrend (potential rise).
    #          If the MAMA is below the FAMA, it indicates a downtrend (potential fall).
    
    # Moving Average with Variable Period: Smooths price data with variable periods
    df['MAVP'] = talib.MAVP(df['Close'], df['Volume'], minperiod=2, maxperiod=30, matype=0)
    # Value range: [negative infinity, positive infinity]
    # Example: If the close price is above the MAVP, it indicates an uptrend (potential rise).
    #          If the close price is below the MAVP, it indicates a downtrend (potential fall).
    
    # MidPoint over Period: Average of the highest and lowest prices
    df['MIDPOINT'] = talib.MIDPOINT(df['Close'], timeperiod=14)
    # Value range: [negative infinity, positive infinity]
    # Example: If the close price is above the MIDPOINT, it indicates an uptrend (potential rise).
    #          If the close price is below the MIDPOINT, it indicates a downtrend (potential fall).
    
    # Midpoint Price over Period: Average of the highest and lowest prices
    df['MIDPRICE'] = talib.MIDPRICE(df['High'], df['Low'], timeperiod=14)
    # Value range: [negative infinity, positive infinity]
    # Example: If the midpoint price is above the MIDPRICE, it indicates an uptrend (potential rise).
    #          If the midpoint price is below the MIDPRICE, it indicates a downtrend (potential fall).
    
    # Parabolic SAR: Identifies potential reversal points
    df['SAR'] = talib.SAR(df['High'], df['Low'], acceleration=0.02, maximum=0.2)
    # Value range: [negative infinity, positive infinity]
    # Example: If the price is above the SAR, it indicates an uptrend (potential rise).
    #          If the price is below the SAR, it indicates a downtrend (potential fall).
    
    # Parabolic SAR - Extended: Identifies potential reversal points with extended parameters
    df['SAREXT'] = talib.SAREXT(df['High'], df['Low'], startvalue=0, offsetonreverse=0, accelerationinitlong=0.02, accelerationlong=0.02, accelerationmaxlong=0.2, accelerationinitshort=0.02, accelerationshort=0.02, accelerationmaxshort=0.2)
    # Value range: [negative infinity, positive infinity]
    # Example: If the price is above the SAREXT, it indicates an uptrend (potential rise).
    #          If the price is below the SAREXT, it indicates a downtrend (potential fall).
    
    # Simple Moving Average: Smooths price data
    df['SMA'] = talib.SMA(df['Close'], timeperiod=30)
    # Value range: [negative infinity, positive infinity]
    # Example: If the close price is above the SMA, it indicates an uptrend (potential rise).
    #          If the close price is below the SMA, it indicates a downtrend (potential fall).
    
    # Triple Exponential Moving Average (T3): Smooths price data with less lag
    df['T3'] = talib.T3(df['Close'], timeperiod=5, vfactor=0.7)
    # Value range: [negative infinity, positive infinity]
    # Example: If the close price is above the T3, it indicates an uptrend (potential rise).
    #          If the close price is below the T3, it indicates a downtrend (potential fall).
    
    # Triple Exponential Moving Average: Smooths price data
    df['TEMA'] = talib.TEMA(df['Close'], timeperiod=30)
    # Value range: [negative infinity, positive infinity]
    # Example: If the close price is above the TEMA, it indicates an uptrend (potential rise).
    #          If the close price is below the TEMA, it indicates a downtrend (potential fall).
    
    # Triangular Moving Average: Smooths price data
    df['TRIMA'] = talib.TRIMA(df['Close'], timeperiod=30)
    # Value range: [negative infinity, positive infinity]
    # Example: If the close price is above the TRIMA, it indicates an uptrend (potential rise).
    #          If the close price is below the TRIMA, it indicates a downtrend (potential fall).
    
    # Weighted Moving Average: Smooths price data
    df['WMA'] = talib.WMA(df['Close'], timeperiod=30)
    # Value range: [negative infinity, positive infinity]
    # Example: If the close price is above the WMA, it indicates an uptrend (potential rise).
    #          If the close price is below the WMA, it indicates a downtrend (potential fall).

    # Momentum Indicators

    # Average Directional Movement Index: Measures trend strength
    df['ADX'] = talib.ADX(df['High'], df['Low'], df['Close'], timeperiod=14)
    # Value range: [0, 100]
    # Example: Higher ADX values indicate a stronger trend. Values above 25 suggest a strong trend.
    
    # Average Directional Movement Index Rating: Measures trend strength
    df['ADXR'] = talib.ADXR(df['High'], df['Low'], df['Close'], timeperiod=14)
    # Value range: [0, 100]
    # Example: Higher ADXR values indicate a stronger trend. Values above 25 suggest a strong trend.
    
    # Absolute Price Oscillator: Measures momentum
    df['APO'] = talib.APO(df['Close'], fastperiod=12, slowperiod=26, matype=0)
    # Value range: [negative infinity, positive infinity]
    # Example: Positive APO values indicate upward momentum. Negative APO values indicate downward momentum.
    
    # Aroon: Identifies trend changes
    df['AROON_down'], df['AROON_up'] = talib.AROON(df['High'], df['Low'], timeperiod=14)
    # Value range: [0, 100]
    # Example: Higher AROON_up values indicate a stronger uptrend. Higher AROON_down values indicate a stronger downtrend.
    
    # Aroon Oscillator: Measures trend strength
    df['AROONOSC'] = talib.AROONOSC(df['High'], df['Low'], timeperiod=14)
    # Value range: [-100, 100]
    # Example: Positive AROONOSC values indicate upward momentum. Negative AROONOSC values indicate downward momentum.
    
    # Balance Of Power: Measures buying and selling pressure
    df['BOP'] = talib.BOP(df['Open'], df['High'], df['Low'], df['Close'])
    # Value range: [-1, 1]
    # Example: Positive BOP values indicate buying pressure. Negative BOP values indicate selling pressure.
    
    # Commodity Channel Index: Identifies cyclical trends
    df['CCI'] = talib.CCI(df['High'], df['Low'], df['Close'], timeperiod=14)
    # Value range: [negative infinity, positive infinity]
    # Example: CCI values above 100 indicate overbought conditions. CCI values below -100 indicate oversold conditions.
    
    # Chande Momentum Oscillator: Measures momentum
    df['CMO'] = talib.CMO(df['Close'], timeperiod=14)
    # Value range: [-100, 100]
    # Example: Positive CMO values indicate upward momentum. Negative CMO values indicate downward momentum.
    
    # Directional Movement Index: Measures trend strength
    df['DX'] = talib.DX(df['High'], df['Low'], df['Close'], timeperiod=14)
    # Value range: [0, 100]
    # Example: Higher DX values indicate a stronger trend. Values above 25 suggest a strong trend.
    
    # Moving Average Convergence/Divergence: Measures momentum
    df['MACD'], df['MACD_signal'], df['MACD_hist'] = talib.MACD(df['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
    # Value range: [negative infinity, positive infinity]
    # Example: Positive MACD values indicate upward momentum. Negative MACD values indicate downward momentum.
    
    # MACD with controllable MA type: Measures momentum
    df['MACDEXT'], df['MACDEXT_signal'], df['MACDEXT_hist'] = talib.MACDEXT(df['Close'], fastperiod=12, fastmatype=0, slowperiod=26, slowmatype=0, signalperiod=9, signalmatype=0)
    # Value range: [negative infinity, positive infinity]
    # Example: Positive MACDEXT values indicate upward momentum. Negative MACDEXT values indicate downward momentum.
    
    # MACD Fix 12/26: Measures momentum
    df['MACDFIX'], df['MACDFIX_signal'], df['MACDFIX_hist'] = talib.MACDFIX(df['Close'], signalperiod=9)
    # Value range: [negative infinity, positive infinity]
    # Example: Positive MACDFIX values indicate upward momentum. Negative MACDFIX values indicate downward momentum.
    
    # Money Flow Index: Measures buying and selling pressure
    df['MFI'] = talib.MFI(df['High'], df['Low'], df['Close'], df['Volume'], timeperiod=14)
    # Value range: [0, 100]
    # Example: MFI values above 80 indicate overbought conditions. MFI values below 20 indicate oversold conditions.
    
    # Minus Directional Indicator: Measures trend strength
    df['MINUS_DI'] = talib.MINUS_DI(df['High'], df['Low'], df['Close'], timeperiod=14)
    # Value range: [0, 100]
    # Example: Higher MINUS_DI values indicate a stronger downtrend.
    
    # Minus Directional Movement: Measures trend strength
    df['MINUS_DM'] = talib.MINUS_DM(df['High'], df['Low'], timeperiod=14)
    # Value range: [negative infinity, positive infinity]
    # Example: Positive MINUS_DM values indicate downward movement.
    
    # Momentum: Measures momentum
    df['MOM'] = talib.MOM(df['Close'], timeperiod=10)
    # Value range: [negative infinity, positive infinity]
    # Example: Positive MOM values indicate upward momentum. Negative MOM values indicate downward momentum.
    
    # Plus Directional Indicator: Measures trend strength
    df['PLUS_DI'] = talib.PLUS_DI(df['High'], df['Low'], df['Close'], timeperiod=14)
    # Value range: [0, 100]
    # Example: Higher PLUS_DI values indicate a stronger uptrend.
    
    # Plus Directional Movement: Measures trend strength
    df['PLUS_DM'] = talib.PLUS_DM(df['High'], df['Low'], timeperiod=14)
    # Value range: [negative infinity, positive infinity]
    # Example: Positive PLUS_DM values indicate upward movement.
    
    # Percentage Price Oscillator: Measures momentum
    df['PPO'] = talib.PPO(df['Close'], fastperiod=12, slowperiod=26, matype=0)
    # Value range: [negative infinity, positive infinity]
    # Example: Positive PPO values indicate upward momentum. Negative PPO values indicate downward momentum.
    
    # Rate of Change: Measures rate of change
    df['ROC'] = talib.ROC(df['Close'], timeperiod=10)
    # Value range: [negative infinity, positive infinity]
    # Example: Positive ROC values indicate upward momentum. Negative ROC values indicate downward momentum.
    
    # Rate of Change Percentage: Measures rate of change percentage
    df['ROCP'] = talib.ROCP(df['Close'], timeperiod=10)
    # Value range: [negative infinity, positive infinity]
    # Example: Positive ROCP values indicate upward momentum. Negative ROCP values indicate downward momentum.

    # Rate of Change Ratio: Measures rate of change ratio
    df['ROCR'] = talib.ROCR(df['Close'], timeperiod=10)
    # Value range: [negative infinity, positive infinity]
    # Example: Positive ROCR values indicate upward momentum. Negative ROCR values indicate downward momentum.
    
    # Rate of Change Ratio 100 Scale: Measures rate of change ratio scaled by 100
    df['ROCR100'] = talib.ROCR100(df['Close'], timeperiod=10)
    # Value range: [negative infinity, positive infinity]
    # Example: Positive ROCR100 values indicate upward momentum. Negative ROCR100 values indicate downward momentum.
    
    # Relative Strength Index: Measures momentum
    df['RSI'] = talib.RSI(df['Close'], timeperiod=14)
    # Value range: [0, 100]
    # Example: RSI values above 70 indicate overbought conditions (potential fall). RSI values below 30 indicate oversold conditions (potential rise).
    
    # Stochastic: Measures momentum
    df['STOCH_slowk'], df['STOCH_slowd'] = talib.STOCH(df['High'], df['Low'], df['Close'], fastk_period=14, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
    # Value range: [0, 100]
    # Example: Stochastic values above 80 indicate overbought conditions (potential fall). Stochastic values below 20 indicate oversold conditions (potential rise).
    
    # Stochastic Fast: Measures momentum
    df['STOCHF_fastk'], df['STOCHF_fastd'] = talib.STOCHF(df['High'], df['Low'], df['Close'], fastk_period=14, fastd_period=3, fastd_matype=0)
    # Value range: [0, 100]
    # Example: Fast Stochastic values above 80 indicate overbought conditions (potential fall). Fast Stochastic values below 20 indicate oversold conditions (potential rise).
    
    # Stochastic Relative Strength Index: Measures momentum
    df['STOCHRSI_fastk'], df['STOCHRSI_fastd'] = talib.STOCHRSI(df['Close'], timeperiod=14, fastk_period=14, fastd_period=3, fastd_matype=0)
    # Value range: [0, 100]
    # Example: Stochastic RSI values above 80 indicate overbought conditions (potential fall). Stochastic RSI values below 20 indicate oversold conditions (potential rise).
    
    # TRIX: Measures rate of change of a triple smoothed EMA
    df['TRIX'] = talib.TRIX(df['Close'], timeperiod=30)
    # Value range: [negative infinity, positive infinity]
    # Example: Positive TRIX values indicate upward momentum. Negative TRIX values indicate downward momentum.
    
    # Ultimate Oscillator: Measures momentum
    df['ULTOSC'] = talib.ULTOSC(df['High'], df['Low'], df['Close'], timeperiod1=7, timeperiod2=14, timeperiod3=28)
    # Value range: [0, 100]
    # Example: ULTOSC values above 70 indicate overbought conditions (potential fall). ULTOSC values below 30 indicate oversold conditions (potential rise).
    
    # Williams' %R: Measures overbought/oversold conditions
    df['WILLR'] = talib.WILLR(df['High'], df['Low'], df['Close'], timeperiod=14)
    # Value range: [-100, 0]
    # Example: WILLR values above -20 indicate overbought conditions (potential fall). WILLR values below -80 indicate oversold conditions (potential rise).
    
    # Volume Indicators
    
    # Chaikin A/D Line: Measures accumulation/distribution
    df['AD'] = talib.AD(df['High'], df['Low'], df['Close'], df['Volume'])
    # Value range: [negative infinity, positive infinity]
    # Example: Positive AD values indicate accumulation (buying pressure). Negative AD values indicate distribution (selling pressure).
    
    # Chaikin A/D Oscillator: Measures momentum of the A/D line
    df['ADOSC'] = talib.ADOSC(df['High'], df['Low'], df['Close'], df['Volume'], fastperiod=3, slowperiod=10)
    # Value range: [negative infinity, positive infinity]
    # Example: Positive ADOSC values indicate upward momentum. Negative ADOSC values indicate downward momentum.
    
    # On Balance Volume: Measures buying and selling pressure
    df['OBV'] = talib.OBV(df['Close'], df['Volume'])
    # Value range: [negative infinity, positive infinity]
    # Example: Positive OBV values indicate buying pressure. Negative OBV values indicate selling pressure.

    #Cycle Indicators

    # Hilbert Transform - Dominant Cycle Period: Identifies dominant cycle period
    df['HT_DCPERIOD'] = talib.HT_DCPERIOD(df['Close'])
    # Value range: [negative infinity, positive infinity]
    # Example: Higher HT_DCPERIOD values indicate a longer dominant cycle period.
    
    # Hilbert Transform - Dominant Cycle Phase: Identifies dominant cycle phase
    df['HT_DCPHASE'] = talib.HT_DCPHASE(df['Close'])
    # Value range: [negative infinity, positive infinity]
    # Example: HT_DCPHASE values oscillate between 0 and 360 degrees, indicating the phase of the dominant cycle.
    
    # Hilbert Transform - Phasor Components: Identifies phasor components
    df['HT_PHASOR_inphase'], df['HT_PHASOR_quadrature'] = talib.HT_PHASOR(df['Close'])
    # Value range: [negative infinity, positive infinity]
    # Example: Inphase and quadrature components help identify the position within the cycle.
    
    # Hilbert Transform - SineWave: Identifies sinewave components
    df['HT_SINE_sine'], df['HT_SINE_leadsine'] = talib.HT_SINE(df['Close'])
    # Value range: [negative infinity, positive infinity]
    # Example: Sine and leadsine values help identify turning points in the cycle.
    
    # Hilbert Transform - Trend vs Cycle Mode: Identifies trend vs cycle mode
    df['HT_TRENDMODE'] = talib.HT_TRENDMODE(df['Close'])
    # Value range: [0, 1]
    # Example: A value of 1 indicates a trending market. A value of 0 indicates a cyclical market.
    
    # Price Transform
    
    # Average Price: Calculates average price
    df['AVGPRICE'] = talib.AVGPRICE(df['Open'], df['High'], df['Low'], df['Close'])
    # Value range: [negative infinity, positive infinity]
    # Example: The average price is calculated as (Open + High + Low + Close) / 4.
    
    # Median Price: Calculates median price
    df['MEDPRICE'] = talib.MEDPRICE(df['High'], df['Low'])
    # Value range: [negative infinity, positive infinity]
    # Example: The median price is calculated as (High + Low) / 2.
    
    # Typical Price: Calculates typical price
    df['TYPPRICE'] = talib.TYPPRICE(df['High'], df['Low'], df['Close'])
    # Value range: [negative infinity, positive infinity]
    # Example: The typical price is calculated as (High + Low + Close) / 3.
    
    # Weighted Close Price: Calculates weighted close price
    df['WCLPRICE'] = talib.WCLPRICE(df['High'], df['Low'], df['Close'])
    # Value range: [negative infinity, positive infinity]
    # Example: The weighted close price is calculated as (High + Low + 2 * Close) / 4.
    
    # Volatility Indicators
    
    # Average True Range: Measures volatility
    df['ATR'] = talib.ATR(df['High'], df['Low'], df['Close'], timeperiod=14)
    # Value range: [negative infinity, positive infinity]
    # Example: Higher ATR values indicate higher volatility.
    
    # Normalized Average True Range: Measures normalized volatility
    df['NATR'] = talib.NATR(df['High'], df['Low'], df['Close'], timeperiod=14)
    # Value range: [negative infinity, positive infinity]
    # Example: Higher NATR values indicate higher normalized volatility.
    
    # True Range: Measures true range
    df['TRANGE'] = talib.TRANGE(df['High'], df['Low'], df['Close'])
    # Value range: [negative infinity, positive infinity]
    # Example: Higher TRANGE values indicate a larger range between high, low, and close prices.

    # Pattern Recognition

    # Get all candlestick pattern functions
    all_functions = talib.get_function_groups()
    candlestick_patterns = all_functions['Pattern Recognition']
    patterns = {pattern: getattr(talib, pattern) for pattern in candlestick_patterns}

    # Initialize Pattern_Sum column
    df['PATTERN_SUM'] = 0

    # Apply each pattern function to the DataFrame and sum the results
    for pattern_name, pattern_func in patterns.items():
        pattern_result = pattern_func(df['Open'], df['High'], df['Low'], df['Close'])
        df['PATTERN_SUM'] += pattern_result

    # Normalize the summed pattern values to be within the range of -1 to 1
    df['PATTERN_SUM'] = df['PATTERN_SUM'].apply(lambda x: np.clip(x, -100, 100) / 100)
    # Value range: [-1, 1]
    # Example: -1 bearish , 0 no detection, 1 bullish
        
    # Beta: Measures volatility relative to the market
    df['BETA'] = talib.BETA(df['High'], df['Low'], timeperiod=5)
    # Value range: [negative infinity, positive infinity]
    # Example: A BETA value greater than 1 indicates higher volatility relative to the market. A BETA value less than 1 indicates lower volatility.
    
    # Pearson's Correlation Coefficient (r): Measures correlation
    df['CORREL'] = talib.CORREL(df['High'], df['Low'], timeperiod=30)
    # Value range: [-1, 1]
    # Example: A CORREL value close to 1 indicates a strong positive correlation. A CORREL value close to -1 indicates a strong negative correlation.
    
    # Linear Regression: Calculates linear regression
    df['LINEARREG'] = talib.LINEARREG(df['Close'], timeperiod=14)
    # Value range: [negative infinity, positive infinity]
    # Example: The LINEARREG value represents the predicted close price based on the linear regression model.
    
    # Linear Regression Angle: Calculates linear regression angle
    df['LINEARREG_ANGLE'] = talib.LINEARREG_ANGLE(df['Close'], timeperiod=14)
    # Value range: [negative infinity, positive infinity]
    # Example: A positive LINEARREG_ANGLE indicates an upward trend. A negative LINEARREG_ANGLE indicates a downward trend.
    
    # Linear Regression Intercept: Calculates linear regression intercept
    df['LINEARREG_INTERCEPT'] = talib.LINEARREG_INTERCEPT(df['Close'], timeperiod=14)
    # Value range: [negative infinity, positive infinity]
    # Example: The LINEARREG_INTERCEPT value represents the intercept of the linear regression line with the y-axis.
    
    # Linear Regression Slope: Calculates linear regression slope
    df['LINEARREG_SLOPE'] = talib.LINEARREG_SLOPE(df['Close'], timeperiod=14)
    # Value range: [negative infinity, positive infinity]
    # Example: A positive LINEARREG_SLOPE indicates an upward trend. A negative LINEARREG_SLOPE indicates a downward trend.
    
    # Standard Deviation: Measures volatility
    df['STDDEV'] = talib.STDDEV(df['Close'], timeperiod=5, nbdev=1)
    # Value range: [0, positive infinity]
    # Example: Higher STDDEV values indicate higher volatility.
    
    # Time Series Forecast: Forecasts future values
    df['TSF'] = talib.TSF(df['Close'], timeperiod=14)
    # Value range: [negative infinity, positive infinity]
    # Example: The TSF value represents the forecasted close price based on the time series model.
    
    # Variance: Measures volatility
    df['VAR'] = talib.VAR(df['Close'], timeperiod=5, nbdev=1)
    # Value range: [0, positive infinity]
    # Example: Higher VAR values indicate higher volatility.

    return df

def set_target(df):
    # Create new columns in a separate DataFrame
    new_columns = pd.DataFrame(index=df.index)
    
    # Shift the 'Close' column by one to get the next day's close price
    new_columns['NEXT_DAY_CLOSEPRICE'] = df['Close'].shift(-1)
    
    # Calculate the change in close price from one day to the next
    new_columns['DAILY_CLOSEPRICE_CHANGE'] = new_columns['NEXT_DAY_CLOSEPRICE'] - df['Close']

    # Determine the direction of the close price change
    new_columns['CLOSEPRICE_DIRECTION'] = np.sign(new_columns['DAILY_CLOSEPRICE_CHANGE'])
    
    # Calculate the daily mid price as the average of the high and low prices
    new_columns['DAILY_MIDPRICE'] = (df['High'] + df['Low']) / 2
    
    # Shift the 'DAILY_MIDPRICE' column by one to get the next day's mid price
    new_columns['NEXT_DAY_MIDPRICE'] = new_columns['DAILY_MIDPRICE'].shift(-1)
    
    # Calculate the change in mid price from one day to the next
    new_columns['DAILY_MIDPRICE_CHANGE'] = new_columns['NEXT_DAY_MIDPRICE'] - new_columns['DAILY_MIDPRICE']

    # Determine the direction of the mid price change
    new_columns['MIDPRICE_DIRECTION'] = np.sign(new_columns['DAILY_MIDPRICE_CHANGE'])
    
    # Handle the last row where changes are NaN
    new_columns.at[df.index[-1], 'DAILY_CLOSEPRICE_CHANGE'] = np.nan
    new_columns.at[df.index[-1], 'CLOSEPRICE_DIRECTION'] = np.nan
    new_columns.at[df.index[-1], 'DAILY_MIDPRICE_CHANGE'] = np.nan
    new_columns.at[df.index[-1], 'MIDPRICE_DIRECTION'] = np.nan
    
    # Concatenate the new columns with the original DataFrame
    df = pd.concat([df, new_columns], axis=1)
    
    return df

# Explanation:
# - Close Price: Used for end-of-day analysis and long-term trend predictions. It is stable and widely used in technical indicators.
# - Mid Price: Useful for intraday analysis and understanding market liquidity. It provides a more granular view of price movements within the trading day.
# - Price Differences: Using differences (e.g., DAILY_CLOSEPRICE_CHANGE) instead of actual prices (e.g., NEXT_DAY_CLOSEPRICE) helps in normalizing the data, making it more comparable across different stocks and time periods. 
#   It also simplifies trend analysis and can improve model performance by focusing on the direction and magnitude of price changes.

In [9]:
def remove_last_two_words(text):
    words = text.split()
    return ' '.join(words[:-2])

# Function to parse date from feed entry
def parse_date(entry):
    return datetime(*entry.published_parsed[:6])

def analyze_sentiment(text):
  
    words = word_tokenize(text.upper())
    
    sentiment_scores = {
        'SENTIMENT_NEGATIVE': 0,
        'SENTIMENT_POSITIVE': 0,
        'SENTIMENT_UNCERTAINTY': 0,
        'SENTIMENT_LITIGIOUS': 0,
        'SENTIMENT_STRONG_MODAL': 0,
        'SENTIMENT_WEAK_MODAL': 0,
        'SENTIMENT_CONSTRAINING': 0
    }
    
    for word in words:
        if word in negative_words:
            sentiment_scores['SENTIMENT_NEGATIVE'] += 1
        if word in positive_words:
            sentiment_scores['SENTIMENT_POSITIVE'] += 1
        if word in uncertainty_words:
            sentiment_scores['SENTIMENT_UNCERTAINTY'] += 1
        if word in litigious_words:
            sentiment_scores['SENTIMENT_LITIGIOUS'] += 1
        if word in strong_modal_words:
            sentiment_scores['SENTIMENT_STRONG_MODAL'] += 1
        if word in weak_modal_words:
            sentiment_scores['SENTIMENT_WEAK_MODAL'] += 1
        if word in constraining_words:
            sentiment_scores['SENTIMENT_CONSTRAINING'] += 1
    print(f"{text} : {sentiment_scores}.")
    return sentiment_scores

# Function to calculate sentiment score
def fetch_and_analyze(query, start_date, end_date):
    encoded_query = quote(query)
    rss_url = f"https://news.google.com/rss/search?q={encoded_query}&hl=en-US&gl=US&ceid=US:en"
    feed = feedparser.parse(rss_url)
    
    sentiment_scores = {
        'SENTIMENT_NEGATIVE': 0,
        'SENTIMENT_POSITIVE': 0,
        'SENTIMENT_UNCERTAINTY': 0,
        'SENTIMENT_LITIGIOUS': 0,
        'SENTIMENT_STRONG_MODAL': 0,
        'SENTIMENT_WEAK_MODAL': 0,
        'SENTIMENT_CONSTRAINING': 0
    }

    for entry in feed.entries:
      
        entry_date = parse_date(entry)
        if start_date <= entry_date <= end_date:
            sentiment = analyze_sentiment(entry.title)
            for key in sentiment_scores:
                sentiment_scores[key] += sentiment[key]
        
    if len(feed.entries) > 0:
        for key in sentiment_scores:
            sentiment_scores[key] /= len(feed.entries)
    
    return sentiment_scores

def get_new_sentiment_score(df, search, batch_size=300):
    # Define the sentiment columns to be added to the DataFrame
    sentiment_columns = [
        'SENTIMENT_NEGATIVE', 'SENTIMENT_POSITIVE', 'SENTIMENT_UNCERTAINTY',
        'SENTIMENT_LITIGIOUS', 'SENTIMENT_STRONG_MODAL', 'SENTIMENT_WEAK_MODAL',
        'SENTIMENT_CONSTRAINING'
    ]
    
    # Initialize sentiment columns with float dtype to avoid type issues
    for col in sentiment_columns:
        df[col] = 0.0
    
    # Cache to store already computed sentiment scores to avoid redundant calculations
    cache = {}
    
    # Use ThreadPoolExecutor to parallelize the fetching and analyzing of sentiment scores
    with ThreadPoolExecutor() as executor:
        # Process the DataFrame in batches to manage memory and performance
        for start in range(0, len(df), batch_size):
            end = min(start + batch_size, len(df))
            batch_df = df.iloc[start:end]
            
            futures = []
            # Iterate over the batch to compute sentiment scores for each row
            for i in range(1, len(batch_df)):
                current_row = batch_df.iloc[i]
                previous_row = batch_df.iloc[i - 1]
                
                start_date = previous_row.name
                end_date = current_row.name
                
                # Iterate over each search query to fetch and analyze sentiment
                for query in search:
                    cache_key = (query, start_date, end_date)
                    if cache_key in cache:
                        sentiment_scores = cache[cache_key]
                    else:
                        # Submit the fetch_and_analyze task to the executor
                        future = executor.submit(fetch_and_analyze, query, start_date, end_date)
                        futures.append((future, cache_key))
            
            # Collect results from futures and update the DataFrame
            for future, cache_key in futures:
                result = future.result()
                cache[cache_key] = result
                for key in sentiment_columns:
                    df.at[current_row.name, key] += float(result[key])
            
            # Average the sentiment scores over the number of search queries
            for key in sentiment_columns:
                if len(search) > 0:
                    df.loc[batch_df.index, key] /= len(search)
            
            # Debugging statements to verify the processing of each batch
            print(f"Batch {start} to {end} processed")
            print(df.loc[batch_df.index, sentiment_columns])
    
    return df

In [10]:
# Check if the ticker symbol file exists
if not os.path.isfile(ticker_symbol_file):
    print(f"Ticker symbol file '{ticker_symbol_file}' does not exist.")
else:
    # Ensure the data directory exists
    os.makedirs('../data', exist_ok=True)

    # Read ticker symbols from file
    with open(ticker_symbol_file, 'r') as file:
        ticker_symbols = file.readlines()

    ticker_symbol_list = [ticker_symbol.strip() for ticker_symbol in ticker_symbols]
    end_date = datetime.today()
    start_date = end_date - timedelta(days=1*365)

    for ticker_symbol in ticker_symbol_list:
        ticker = ticker_symbol
        try:
            df = yf.download(ticker, start=start_date, end=end_date)
            
            stock = yf.Ticker(ticker)
            info = stock.info
            news_search = []
            news_search.append(ticker)
            
            ticker_type = info.get('quoteType', 'Unknown')
            if ticker_type == 'EQUITY':
                company_name = info.get('shortName', '')
                #industry = info.get('industry', '')
                #sector = info.get('sector', '')
                news_search.append(company_name)
                #news_search.append(industry)
                #news_search.append(sector)
            elif ticker_type == 'CURRENCY':
                currency_name = info.get('currency', '')
                news_search.append(currency_name)
            elif ticker_type == 'FUTURE':
                commodity_name = info.get('shortName', '')
                commodity_name = remove_last_two_words(commodity_name)
                news_search.append(commodity_name)
            
        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")
            continue
        df = get_technical_indicator(df)
        df = get_new_sentiment_score(df, news_search)
        df = set_target(df)
        df.to_csv(f'../data/{ticker}.csv')


[*********************100%%**********************]  1 of 1 completed


NVIDIA Blackwell Platform Arrives to Power a New Era of Computing - NVIDIA Blog : {'SENTIMENT_NEGATIVE': 0, 'SENTIMENT_POSITIVE': 0, 'SENTIMENT_UNCERTAINTY': 0, 'SENTIMENT_LITIGIOUS': 0, 'SENTIMENT_STRONG_MODAL': 0, 'SENTIMENT_WEAK_MODAL': 0, 'SENTIMENT_CONSTRAINING': 0}.
NVIDIA Announces Financial Results for First Quarter Fiscal 2025 - NVIDIA Blog : {'SENTIMENT_NEGATIVE': 0, 'SENTIMENT_POSITIVE': 0, 'SENTIMENT_UNCERTAINTY': 0, 'SENTIMENT_LITIGIOUS': 0, 'SENTIMENT_STRONG_MODAL': 0, 'SENTIMENT_WEAK_MODAL': 0, 'SENTIMENT_CONSTRAINING': 0}.
AI Solutions for Enterprises | NVIDIA - NVIDIA Daily News Report : {'SENTIMENT_NEGATIVE': 0, 'SENTIMENT_POSITIVE': 0, 'SENTIMENT_UNCERTAINTY': 0, 'SENTIMENT_LITIGIOUS': 0, 'SENTIMENT_STRONG_MODAL': 0, 'SENTIMENT_WEAK_MODAL': 0, 'SENTIMENT_CONSTRAINING': 0}.
Nvidia Second Quarter Earnings Preview: What To Know Before The AI Leader Reports - Forbes : {'SENTIMENT_NEGATIVE': 0, 'SENTIMENT_POSITIVE': 0, 'SENTIMENT_UNCERTAINTY': 0, 'SENTIMENT_LITIGIOUS': 0,

[*********************100%%**********************]  1 of 1 completed


SGD Stock Price and Chart — NASDAQ:SGD - TradingView : {'SENTIMENT_NEGATIVE': 0, 'SENTIMENT_POSITIVE': 0, 'SENTIMENT_UNCERTAINTY': 0, 'SENTIMENT_LITIGIOUS': 0, 'SENTIMENT_STRONG_MODAL': 0, 'SENTIMENT_WEAK_MODAL': 0, 'SENTIMENT_CONSTRAINING': 0}.
Safe & Green Holdings Corp. Completes Spin-Off of Safe and Green Development Corporation - Business Wire : {'SENTIMENT_NEGATIVE': 0, 'SENTIMENT_POSITIVE': 0, 'SENTIMENT_UNCERTAINTY': 0, 'SENTIMENT_LITIGIOUS': 0, 'SENTIMENT_STRONG_MODAL': 0, 'SENTIMENT_WEAK_MODAL': 0, 'SENTIMENT_CONSTRAINING': 0}.
Protos Labs Raises SGD $3 Million in Oversubscribed Seed Round to Revolutionize Cyber Risk Management - Yahoo Finance : {'SENTIMENT_NEGATIVE': 0, 'SENTIMENT_POSITIVE': 1, 'SENTIMENT_UNCERTAINTY': 1, 'SENTIMENT_LITIGIOUS': 0, 'SENTIMENT_STRONG_MODAL': 0, 'SENTIMENT_WEAK_MODAL': 0, 'SENTIMENT_CONSTRAINING': 0}.
SGD/MYR: Singapore Dollar at All-Time High Against Ringgit on Policy Gap - Bloomberg : {'SENTIMENT_NEGATIVE': 1, 'SENTIMENT_POSITIVE': 0, 'SENTIM

[*********************100%%**********************]  1 of 1 completed

CIMB Singapore offers 3.55% p.a. rate on new SGD business fixed deposit accounts - Yahoo Finance : {'SENTIMENT_NEGATIVE': 0, 'SENTIMENT_POSITIVE': 0, 'SENTIMENT_UNCERTAINTY': 0, 'SENTIMENT_LITIGIOUS': 0, 'SENTIMENT_STRONG_MODAL': 0, 'SENTIMENT_WEAK_MODAL': 0, 'SENTIMENT_CONSTRAINING': 0}.
Analysts say Singapore dollar could weaken if US economy can avoid recession in the months ahead - The Independent : {'SENTIMENT_NEGATIVE': 2, 'SENTIMENT_POSITIVE': 0, 'SENTIMENT_UNCERTAINTY': 1, 'SENTIMENT_LITIGIOUS': 0, 'SENTIMENT_STRONG_MODAL': 0, 'SENTIMENT_WEAK_MODAL': 1, 'SENTIMENT_CONSTRAINING': 0}.
Batch 0 to 261 processed
            SENTIMENT_NEGATIVE  SENTIMENT_POSITIVE  SENTIMENT_UNCERTAINTY  \
Date                                                                        
2023-09-04            0.000000            0.000000                0.00000   
2023-09-05            0.000000            0.000000                0.00000   
2023-09-06            0.000000            0.000000                0.0

12 Money Changers In Singapore Near You To Get The Best Exchange Rates - TheSmartLocal : {'SENTIMENT_NEGATIVE': 0, 'SENTIMENT_POSITIVE': 1, 'SENTIMENT_UNCERTAINTY': 0, 'SENTIMENT_LITIGIOUS': 0, 'SENTIMENT_STRONG_MODAL': 1, 'SENTIMENT_WEAK_MODAL': 0, 'SENTIMENT_CONSTRAINING': 0}.
Explainer: How Singapore's unique monetary policy works - Reuters : {'SENTIMENT_NEGATIVE': 0, 'SENTIMENT_POSITIVE': 0, 'SENTIMENT_UNCERTAINTY': 0, 'SENTIMENT_LITIGIOUS': 0, 'SENTIMENT_STRONG_MODAL': 0, 'SENTIMENT_WEAK_MODAL': 0, 'SENTIMENT_CONSTRAINING': 0}.
StraitsX gets green light to issue SGD, USD stablecoins in Singapore - Tech in Asia : {'SENTIMENT_NEGATIVE': 0, 'SENTIMENT_POSITIVE': 0, 'SENTIMENT_UNCERTAINTY': 0, 'SENTIMENT_LITIGIOUS': 0, 'SENTIMENT_STRONG_MODAL': 0, 'SENTIMENT_WEAK_MODAL': 0, 'SENTIMENT_CONSTRAINING': 0}.
Astrea 8 Bonds Review: Interest, Risk and More Info on These Private Equity (PE) Bonds From Temasek-Owned Azalea - Seedly : {'SENTIMENT_NEGATIVE': 0, 'SENTIMENT_POSITIVE': 0, 'SENTIMEN

[*********************100%%**********************]  1 of 1 completed

US Dollar Forecast: Strong support emerges around 100.50 - FXStreet : {'SENTIMENT_NEGATIVE': 0, 'SENTIMENT_POSITIVE': 1, 'SENTIMENT_UNCERTAINTY': 0, 'SENTIMENT_LITIGIOUS': 0, 'SENTIMENT_STRONG_MODAL': 0, 'SENTIMENT_WEAK_MODAL': 0, 'SENTIMENT_CONSTRAINING': 0}.
USD/JPY Weekly Price Forecast – US Dollar Continues to See Support Against Yen - Yahoo Finance : {'SENTIMENT_NEGATIVE': 1, 'SENTIMENT_POSITIVE': 0, 'SENTIMENT_UNCERTAINTY': 0, 'SENTIMENT_LITIGIOUS': 0, 'SENTIMENT_STRONG_MODAL': 0, 'SENTIMENT_WEAK_MODAL': 0, 'SENTIMENT_CONSTRAINING': 0}.
UBS Predicts Further Weakening of US Dollar - Bitcoin.com News : {'SENTIMENT_NEGATIVE': 1, 'SENTIMENT_POSITIVE': 0, 'SENTIMENT_UNCERTAINTY': 1, 'SENTIMENT_LITIGIOUS': 0, 'SENTIMENT_STRONG_MODAL': 0, 'SENTIMENT_WEAK_MODAL': 0, 'SENTIMENT_CONSTRAINING': 0}.
Bankrupt Bitcoin miner Rhodium gets court approval for loan in BTC or USD - Cointelegraph : {'SENTIMENT_NEGATIVE': 1, 'SENTIMENT_POSITIVE': 0, 'SENTIMENT_UNCERTAINTY': 0, 'SENTIMENT_LITIGIOUS': 1

CLF Stock Quote Price and Forecast - CNN : {'SENTIMENT_NEGATIVE': 0, 'SENTIMENT_POSITIVE': 0, 'SENTIMENT_UNCERTAINTY': 0, 'SENTIMENT_LITIGIOUS': 0, 'SENTIMENT_STRONG_MODAL': 0, 'SENTIMENT_WEAK_MODAL': 0, 'SENTIMENT_CONSTRAINING': 0}.
Why Are Steel Stocks X, CLF, NUE, STLD Up Today? - InvestorPlace : {'SENTIMENT_NEGATIVE': 0, 'SENTIMENT_POSITIVE': 0, 'SENTIMENT_UNCERTAINTY': 0, 'SENTIMENT_LITIGIOUS': 0, 'SENTIMENT_STRONG_MODAL': 0, 'SENTIMENT_WEAK_MODAL': 0, 'SENTIMENT_CONSTRAINING': 0}.
10 Top Performing Canadian Mutual Funds in 2023 - Morningstar.ca : {'SENTIMENT_NEGATIVE': 0, 'SENTIMENT_POSITIVE': 0, 'SENTIMENT_UNCERTAINTY': 0, 'SENTIMENT_LITIGIOUS': 0, 'SENTIMENT_STRONG_MODAL': 0, 'SENTIMENT_WEAK_MODAL': 0, 'SENTIMENT_CONSTRAINING': 0}.
Oil prices fall after surprise inventory build in US - Yahoo Finance : {'SENTIMENT_NEGATIVE': 0, 'SENTIMENT_POSITIVE': 0, 'SENTIMENT_UNCERTAINTY': 0, 'SENTIMENT_LITIGIOUS': 0, 'SENTIMENT_STRONG_MODAL': 0, 'SENTIMENT_WEAK_MODAL': 0, 'SENTIMENT_CONSTRA